# Problem statement

100 candidates fill out a form of 10000 questions. Each candidate has a skill $s_i\in(-3,3)$, and each question has a 
difficulty $d_j\in(-3,3)$, both uniformly distributed. The probability for a student with skill $s_i$ to answer correctly 
a question of difficulty $d_j$ is:

$$P(X_{ij}=1 | s_i, d_j) = \sigma(s_i-d_j)$$

where $\sigma: x \to \frac{1}{1+e^{-x}}$ is the sigmoid function.

One of the candidates is a **cheater**: with probability $\frac{1}{2}$, they give the answer that they think is correct, 
but with probability $\frac{1}{2}$ they *cheat* and look up the correct answer on the internet.

By observing only the answers $X$ of all candidates to all questions, without knowing the skills of candidates nor the 
difficulties of questions, we must predict who the cheater is with the highest possible success rate. 

## Define the answering model

If we want to estimate the skills of students and difficulties of questions from the answer data, we must first define
a model describing how the three are related.

Given two arrays of skills and difficulties, `ResponseModel` gives the probability $P(X_{ij}=1 | s_i, d_j)$ that each 
candidate $i$ answers correctly to question $j$:

In [ ]:
import torch
import torch.nn.functional as F
from torch.distributions import Bernoulli
from tqdm import trange
import matplotlib.pyplot as plt

class ResponseModel(torch.nn.Module):
    def __init__(self, n_students, n_questions):
        super().__init__()
        self.skill = torch.nn.Parameter(torch.zeros(n_students))
        self.difficulty = torch.nn.Parameter(torch.zeros(n_questions))

    def forward(self):
        return Bernoulli(probs=torch.sigmoid(self.skill[:, None] - self.difficulty[None, :]))

## Generate some data

We then generate some random skills, difficulties and answers according to this model, and then simulate the presence
of a cheater among the students.  


In [ ]:
def generate_data(n_students, n_questions):
    # Sample students, questions, and answers
    model = ResponseModel(n_students, n_students)
    model.skill.data = -3 + 6*torch.rand(n_students)
    model.difficulty.data = -3 + 6*torch.rand(n_questions)
    answers = model().sample()
    
    # Pick a cheater, and make their answer correct with probability 1/2
    cheater = torch.randint(high=n_students, size=(1,)).item()
    is_cheating = Bernoulli(probs=0.5*torch.ones(n_questions)).sample().bool()
    answers[cheater] = torch.where(is_cheating, torch.ones(n_questions), answers[cheater])
    return model, answers, cheater

true_model, answers, cheater = generate_data(n_students=100, n_questions=10000)
print("Skill", true_model.skill)
print("Difficulty", true_model.difficulty)
print("Answers", answers)

## Fit the model by Maximum Likelihood Estimation

### From MAP to MLE
We need to estimate the skill $s$ of every student and difficulty $d$ of every question, given the observed answers $X$.
The **Maximum A Posteriori (MAP)** estimate is then given by $\text{arg}\max_{s,d} P(s, d | X)$.

By Bayes formula,

$$P(s, d | X) = P(X|s, d)\frac{P(s, d)}{P(X)}$$

From the problem statement, we know that the prior $P(s, d)$ is uniform, so it does not depend on $s,d$, and neither 
does $P(X)$. Thus, the MAP estimate is equal to the **Maximum Likelihood Estimate (MLE)**:

$$\text{arg}\max_{s,d} P(s, d | X) = \text{arg}\max_{s,d}P(X|s, d)$$

And since answers are independent, we have
$P(X|s, d) = \prod_{i,j} P(X_{ij} | s_i, d_j)$

And since $\log$ is increasing, 

$$\text{arg}\max_{s,d}P(X|s, d) = \text{arg}\max_{s,d} \log P(X|s, d) = \text{arg}\max_{s,d}\sum_{i,j} \log P(X_{ij} | s_i, d_j)$$

Thus, the MAP estimate is given my minimising the **negative log-likelihood loss** $\mathcal{L}(s,d) = - \sum_{i,j} \log P(X_{ij} | s_i, d_j)$ of the observed datapoints $X_{ij}$ according to our `ResponseModel`.

### Centering the skills and difficulties

The response model is invariant to translations of skills $s$ and difficulties $d$, and only sensitive to their differences $s-d$.
Thus, we can freely offset them without affecting the induced answers probabilities.
Since the true difficulties and skills are centered around $0$, we can enforce this for our estimates by adding a
term to the loss, penalizing the squared mean difficulty:

$$\mathcal{L}(s,d) = - \sum_{i,j} \log P(X_{ij} | s_i, d_j) + \alpha \left(\sum_j d_j\right)^2$$


In [ ]:
def fit(model, answers, epochs, alpha=1.0, see_progress=True):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    if torch.cuda.is_available():
        model, answers = model.cuda(), answers.cuda()
    losses = []
    for _ in trange(epochs) if see_progress else range(epochs):
        optimizer.zero_grad()
        distribution = model()
        loss = -distribution.log_prob(answers).mean() + alpha*torch.pow(model.difficulty.mean(), 2)
        losses.append(loss.detach())
        loss.backward()
        optimizer.step()
    model, answers = model.cpu(), answers.cpu()
    return model, losses

model = ResponseModel(n_students=answers.shape[0], n_questions=answers.shape[1])
model, losses = fit(model, answers, epochs=1000, alpha=1e-2)
plt.plot(losses, label="loss")
plt.yscale("log")
plt.xlabel("epochs")
plt.legend()

## Visualize estimation errors

In [ ]:
plt.hist((model.difficulty - true_model.difficulty).detach().numpy(), density=True, label="difficulty error")
plt.hist((model.skill - true_model.skill).detach().numpy(), density=True, label="skill error")
plt.legend()

You may notice that only one single student has a skill largely overestimated... Who could that be?

## Predict the cheater, with lowest answers likelihood

In [ ]:
log_probs = model().log_prob(answers).sum(axis=1).detach().numpy()
predicted_cheater = log_probs.argmin()
print(f"Predicted cheater: {predicted_cheater}. True cheater: {cheater}")
print(f"Predicted skill: {model.skill[predicted_cheater]}, true skill: {true_model.skill[cheater]}")

plt.scatter(range(log_probs.size), log_probs, marker='.')
plt.scatter(x=cheater, y=log_probs[cheater], c='r', marker='o', label="true cheater")
plt.scatter(x=predicted_cheater, y=log_probs[predicted_cheater], marker='o', s=80,
            facecolor='none', edgecolor='b', label="predicted cheater")
plt.ylabel("log prob of answers")
plt.xlabel("student")
plt.legend()

## Repeat and measure success rate
*It should be enabled by default, but be sure to select GPU in* `Execution / Execution Type` *to get a faster training time (~4mn).* 

In [ ]:
samples = 100
successes = 0
for _ in trange(samples):
    true_model, answers, cheater = generate_data(n_students=100, n_questions=10000)
    model = ResponseModel(n_students=answers.shape[0], n_questions=answers.shape[1])
    model, _ = fit(model, answers, epochs=1000, alpha=1e-2, see_progress=False)
    predicted_cheater = model().log_prob(answers).sum(axis=1).detach().numpy().argmin()
    successes += predicted_cheater == cheater
print()
print(f"Success rate: {successes/samples*100:.1f}%")


Unfortunately, the success rate is too low, and does not reach the desired target of 84%.
This is because our response model does not account for the presence of the cheater, and its effects on the distribution of answers.

Let us try to include this information in our model.

## Include the cheater in the item response model

We add an additional parameter to the model: the (log-)probability that any student is the cheater.  

In [ ]:
class ResponseModelWithCheater(ResponseModel):
    def __init__(self, n_students, n_questions):
        super().__init__(n_students, n_questions)
        self.cheater_logits = torch.nn.Parameter(torch.zeros(n_students))

    def forward(self):
        cheater_prob = F.softmax(self.cheater_logits)[:, None]
        answer_no_cheat = torch.sigmoid(self.skill[:, None] - self.difficulty[None, :])
        answer_cheat = 0.5 * torch.sigmoid(self.skill[:, None] - self.difficulty[None, :]) + 0.5 * 1
        return Bernoulli(probs=cheater_prob*answer_cheat + (1-cheater_prob)*answer_no_cheat)


## Fit the updated model with cheater

In [ ]:
model = ResponseModelWithCheater(n_students=answers.shape[0], n_questions=answers.shape[1])
model, losses = fit(model, answers, epochs=1000, alpha=1e-2)
plt.plot(losses, label="loss")
plt.yscale("log")
plt.xlabel("epochs")
plt.legend()

## Plot the cheater probabilities, and predicted cheater

In [ ]:
cheater_probs = F.softmax(model.cheater_logits).detach().numpy()
plt.axvline(x=cheater, c='r')
plt.plot(cheater_probs)
plt.xlabel("student")
plt.ylabel("cheater probability")

## Repeat and measure success rate

In [ ]:
samples = 100
successes = 0
for _ in trange(samples):
    true_model, answers, cheater = generate_data(n_students=100, n_questions=10000)
    model = ResponseModelWithCheater(n_students=answers.shape[0], n_questions=answers.shape[1])
    model, _ = fit(model, answers, epochs=1000, alpha=1e-2, see_progress=False)
    predicted_cheater = F.softmax(model.cheater_logits).detach().numpy().argmax()
    successes += predicted_cheater == cheater
print()
print(f"Success rate: {successes/samples*100:.1f}%")
